In [12]:
from transformers import GPT2TokenizerFast
from copy import deepcopy
import numpy as np
from random import random

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

with open("asd.txt", "r") as f:
    all_txt = f.read()
def get_imput(txt, length=512):
    bi = int(random()*(len(txt)-1000))
    return txt[bi:bi+length]

/home/maciej/anaconda3/envs/tokenizex/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
from copy import deepcopy

#dev not working, need positional encoding at the first step to take in to an account!

txt = "wikipediawolna"

def text_emb_pos(txt, tokenizer):
    emb_pos = []

    for i in range(1, len(txt)+1):
        et = tokenizer.encode(txt[:i])
        emb_pos.append(list(zip(et, list(range(len(et))))))

    return emb_pos

def create_mask(emb_pos):
    total_len = len(emb_pos)
    res = []
    mask = np.zeros([total_len, total_len])
    for embt, m in zip(emb_pos, mask):
        res.append(embt[-1])
        m[len(res)-1] = 1
        for ie in embt[:-1]:
            for j, je in enumerate(res[:-1]):
                if je == ie:
                    m[j] = 1
                    break
    return np.array(res), mask

def create_mask_embedding(txt, tokenizer):
    emb_pos = text_emb_pos(txt, tokenizer)
    res, mask = create_mask(emb_pos)    

    return res[:, 0], res[:, 1], mask

emb, pos, mask = create_mask_embedding(txt, tokenizer)


In [14]:
def add_masks(o_mask_size, masks):
    iy = 0
    ix = 0
    o_mask = np.zeros((o_mask_size, o_mask_size))

    for m in masks:
        ms = len(m)
        o_mask[iy:iy+ms, ix:ix+ms] += m
        o_mask[iy+ms:, ix:ix+ms] = m[-1]
        iy+=ms
        ix+=ms
    return o_mask

In [15]:
from functools import cache
from time import time

@cache
def process_token(t, tokenizer):
    td = tokenizer.decode(t)
    if len(td)==1 or "�" in td:
        emb = np.array([t]) 
        pos = np.zeros((1))
        mask = np.ones((1,1))
    else:
        emb, pos, mask = create_mask_embedding(td, tokenizer)
        if len(emb) == 1:
            raise Exception("Used!")
        
    return emb, pos, mask


def embedding_mask(txt, tokenizer):
    tokens = tokenizer.encode(txt)
    partial_masks = []

    res_emb = []
    res_pos = []
    pos_len = 0

    for t in tokens:
        emb, pos, mask = process_token(t, tokenizer)
        res_emb.extend(emb)
        res_pos.extend(pos+pos_len)
        pos_len += pos[-1] + 1
        partial_masks.append(mask)

    return np.array(res_emb), res_pos, add_masks(len(res_emb), partial_masks)

In [16]:
C = 1024

def tokenize_letters(txt, tokenizer):
    emb = []
    for l in txt:
        emb.extend(tokenizer.encode(l))
    return emb

In [ ]:
def tokenize_my():
    emb, pos, mask = embedding_mask(txti, tokenizer)
    return 

In [17]:

tokenizer
st = 0
for i in range(C):
    txti = tokenizer.decode(tokenizer.encode(get_imput(all_txt, 512)))
    t1 = time()
    emb, pos, mask = embedding_mask(txti, tokenizer)
    st += time()-t1
    tok = tokenizer.encode(txti)
    lte = emb[mask[-1].astype(bool)]
    assert lte.tolist() == tok

my = st/C
print(my)



0.0009926934726536274


In [18]:
st = 0
txt = []
for i in range(C):
    txt.append(get_imput(all_txt, 512))
t1 = time()
txti = tokenizer.batch_encode_plus(txt)
st += time()-t1
    
default_batch = st/C
print(default_batch)



1.98091147467494e-05


In [19]:
st = 0
for i in range(C):
    txt = get_imput(all_txt, 512)
    t1 = time()
    txti = tokenizer.encode(txt)
    st += time()-t1
    
default = st/C
print(default)

0.0001425035297870636


In [20]:
def create_atomizer(tokenizer):
    """
    {
        "token_id":{
            "embedding":np.array(int), 
            "pe":np.array(int), 
            "mask":np.array(np.array(union(0, 1)))
        }
    }
    """
    tokenizex_atomizer = {}
    vocab = {v:k for k, v in tokenizer.get_vocab().items()}
    all_tok = vocab.keys()

    for t in all_tok:
        e, p, m = process_token(t, tokenizer)
        tokenizex_atomizer[t] = (e, p, m)
        
        # {
        #     "emb":  e, 
        #     "pe":   p, 
        #     "mask": m
        # }
        
    return tokenizex_atomizer
        
tokenizex_atomizer = create_atomizer(tokenizer)





In [21]:
global st
st=0

def embedding_mask(txt, tokenizer, tokenizex_atomizer):
    global st
    tokens = tokenizer.encode(txt)
    partial_masks = []

    res_emb = []
    res_pos = []
    pos_len = 0

    
    for t in tokens:
        emb, pos, mask = tokenizex_atomizer[t]
        res_emb.extend(emb)
        res_pos.extend(pos+pos_len)
        pos_len += pos[-1] + 1
        partial_masks.append(mask)
                
    
    full_mask = add_masks(len(res_emb), partial_masks)
    
    return np.array(res_emb), res_pos, full_mask


for i in range(C):
    txti = tokenizer.decode(tokenizer.encode(get_imput(all_txt, 512)))

    t1 = time() #dev
    emb, pos, mask = embedding_mask(txti, tokenizer, tokenizex_atomizer)
    t2 = time()    
    st += t2-t1 #dev

    tok = tokenizer.encode(txti)
    lte = emb[mask[-1].astype(bool)]
    assert lte.tolist() == tok

my2 = st/C
print(my2)



0.0009675367036834359


In [22]:
llm_examples = 128*500000

print(f"my1/default ratio {my/default}")
print(f"my2/default ratio {my2/default}")
print(f"my2/default_batch ratio {my2/default_batch}")

my1/default ratio 6.966097430266907
my2/default ratio 6.789563073484433
my2/default_batch ratio 48.84300565941267


In [23]:
tokenizer.vocab_size

50257

In [35]:
txt = "example text to tokenize and detokenize"

def decode(id_s, tokenizer):
    
    emb, pos, mask = embedding_mask(txti, tokenizer, tokenizex_atomizer)




In [36]:
emb, pos, mask = embedding_mask(txt, tokenizer, tokenizex_atomizer)


In [48]:
len(tokenize_letters(txt, tokenizer))


array([   68,  1069,    64,   321,   696,   489, 20688,   220,   256,
         573, 48659,  2420,   220,   256,   284,   220,   256,   284,
          74,   365, 11241,    72,   528,  1096,   220,   257,   281,
         290,   220,   288,   390,  1062,    78,   482,  2088,  4233,
          72,   528,  1096])

In [49]:
len(emb)

39